In [ ]:
import pandas as pd

df = pd.read_csv('start_station_probs.csv')

dictionary = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))
stationlist = []
startProb = []
for station, probs in dictionary.items():
  stationlist.append(station)
  startProb.append(probs)

csv_file_path = "trip_stats.csv"
import csv
probabilities = {}

with open(csv_file_path) as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader) # skip header row
    data = {}
    for row in csv_reader:
        start = row[0]
        end = row[1]
        count = int(row[2])
        if start not in data:
            data[start] = {"end_points": [], "probabilities": []}
        data[start]["end_points"].append(end)
        data[start]["probabilities"].append(count)

    for start, d in data.items():
        total_count = sum(d["probabilities"])
        probabilities[start] = [d["end_points"], [p / total_count for p in d["probabilities"]]]

print(probabilities['South Waterfront Walkway - Sinatra Dr & 1 St'])


[['11 St & Washington St', '12 St & Sinatra Dr N', '14 St Ferry - 14 St & Shipyard Ln', '4 St & Grand St', '5 Corners Library', '6 St & Grand St', '7 St & Monroe St', '8 St & Washington St', '9 St HBLR - Jackson St & 8 St', 'Adams St & 11 St', 'Adams St & 2 St', 'Baldwin at Montgomery', 'Bergen Ave', 'Bergen Ave & Stegman St', 'Bloomfield St & 15 St', 'Brunswick & 6th', 'Brunswick St', 'Christ Hospital', 'Church Sq Park - 5 St & Park Ave', 'City Hall', 'City Hall - Washington St & 1 St', 'Clinton St & 7 St', 'Clinton St & Newark St', 'Columbus Dr at Exchange Pl', 'Columbus Drive', 'Columbus Park - Clinton St & 9 St', 'Dey St', 'Dixon Mills', 'Essex Light Rail', 'Glenwood Ave', 'Grand St', 'Grand St & 14 St', 'Grove St PATH', 'Hamilton Park', 'Harborside', 'Heights Elevator', 'Hilltop', 'Hoboken Ave at Monmouth St', 'Hoboken Terminal - Hudson St & Hudson Pl', 'Hoboken Terminal - River St & Hudson Pl', 'Hudson St & 4 St', 'JC Medical Center', 'Jersey & 3rd', 'Jersey & 6th St', 'Lafayette

In [ ]:
R = 3500 # number of rider

In [ ]:
import random
def randomGenerator():
  result = []
  for _ in range(R):
     i = random.choices(stationlist, weights=startProb)[0]
     j = random.choices(probabilities[i][0], weights=probabilities[i][1])[0]
     result.append([i,j])
  return result

In [ ]:
print(randomGenerator())

[['Jersey & 6th St', 'Jersey & 6th St'], ['Lafayette Park', 'Liberty Light Rail'], ['14 St Ferry - 14 St & Shipyard Ln', '12 St & Sinatra Dr N'], ['Christ Hospital', 'Newport Pkwy'], ['City Hall - Washington St & 1 St', 'Mama Johnson Field - 4 St & Jackson St'], ['Madison St & 1 St', 'South Waterfront Walkway - Sinatra Dr & 1 St'], ['Harborside', 'Columbus Dr at Exchange Pl'], ['Jersey & 3rd', 'Liberty Light Rail'], ['Grand St', '12 St & Sinatra Dr N'], ['12 St & Sinatra Dr N', 'Adams St & 2 St'], ['Lafayette Park', 'Lafayette Park'], ['Manila & 1st', 'Newport PATH'], ['Warren St', '12 St & Sinatra Dr N'], ['Hoboken Terminal - River St & Hudson Pl', '14 St Ferry - 14 St & Shipyard Ln'], ['12 St & Sinatra Dr N', '12 St & Sinatra Dr N'], ['Paulus Hook', 'Dixon Mills'], ['Marin Light Rail', 'Columbus Dr at Exchange Pl'], ['South Waterfront Walkway - Sinatra Dr & 1 St', 'Bloomfield St & 15 St'], ['Baldwin at Montgomery', 'Marin Light Rail'], ['Harborside', 'Liberty Light Rail'], ['Newark A

In [ ]:
from pprint import pprint # pretty-printing basic data structures

In [ ]:
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

In [ ]:
@static_vars(t=0)
def now():
    return now.t

print("The current simulation time is", now(), "o'clock.")

The current simulation time is 0 o'clock.


In [ ]:
def set_time(t_new=0):
    now.t = t_new
    return now()

In [ ]:
from typing import Callable
from functools import total_ordering

@total_ordering
class Event:
    def __init__(self, t: int, i: int, j: int, f: Callable = None):
        self.t = t
        self.i = i
        self.j = j
        self.f = f
      
    def __lt__(self, other):
        if not isinstance(other, Event):
            return NotImplemented
        if self.t < other.t:
            return True
        elif self.t == other.t:
            if self.f.__name__ is not None and other.f.__name__ is not None:
                return self.f.__name__< other.f.__name__  # if more than one Event happens in exactly the same time, set "arrived_startpoint" after "arrived_startpoint" to prevent conflict.
            elif self.f.__name__ is not None:
                return True
        return False

    def __eq__(self, other):
        if not isinstance(other, Event):
            return NotImplemented
        return self.t == other.t and self.i == other.i and self.j == other.j

In [ ]:
class FutureEventList:
    def __init__(self):
        self.events = []
        
    def __iter__(self):
        return self
    
    def __next__(self) -> Event:
        from heapq import heappop
        if self.events:
            return heappop(self.events)
        raise StopIteration
    
    def __repr__(self) -> str:
        from pprint import pformat
        return pformat(self.events)

In [ ]:
def schedule(e: Event, fev: FutureEventList): # inserts `e` into `fev`
    from heapq import heappush
    heappush(fev.events, e)

In [ ]:
def simulate(state, event_list, verbose=True):
    arrived_count = 0
    riding_count = 0 
    alist = []
    
    for e in event_list:
        if e.t > 1440:
            break 
        set_time(e.t)
        e.f(state,e.i,e.j, event_list)
        if e.f.__name__=="arrived_startpoint":
            arrived_count+=1
        if e.f.__name__=="start_riding":
            riding_count+=1
        # if verbose: 
            # print(f"[t={e.t}] [#bikes={state['num_bikes'][e.i]}] [#waiter={len(state['infoWaiter'][e.i])}] [e={e.f.__name__}] [way= [{e.i}] => [{e.j}]]"

    avg_waiting = state['waitTime'] / riding_count
    alist.append(riding_count*100/arrived_count)
    alist.append(avg_waiting)
    # print(f"probability of a successful rental = {riding_count*100/arrived_count:.2f}%   [riders={arrived_count}] [successful rental={riding_count}]")
    # print(f"a rider's average waiting-time = {avg_waiting}min")
    return alist


In [ ]:
def initial_state(num = 10):
    from collections import deque
    num_bikes = {}
    infoWaiter = {}
    for i in stationlist:
      num_bikes[i] =  num  # number of bikes in station i
      infoWaiter[i] = deque([]) # each waiter's destination info; que of [destination,arried_time]
    return {'num_bikes': num_bikes
            ,'infoWaiter': infoWaiter
            ,'waitTime' : 0}


In [ ]:
def arrived_startpoint(s,i,j, fev): 
    s['infoWaiter'][i].append([j, now()])
    if s['num_bikes'][i] > 0:
        schedule(Event(now(),i,j, start_riding), fev)

In [ ]:
def start_riding(s,i,j, fev): 
    assert s['num_bikes'][i] > 0
    assert len(s['infoWaiter'][i]) > 0
    s['num_bikes'][i] -= 1
    arrived_time = s['infoWaiter'][i].popleft()[1]
    s['waitTime'] += now() - arrived_time
    import numpy as np
    mu = 2.78
    sigma = 0.619
    total = 0

    # Generate 10 random ride times
    ride_times = np.random.lognormal(mu, sigma, 10)
    for x in ride_times:
      total += x
    total /= 10
    schedule(Event(now() + total,i,j, arrived_endpoint), fev)
    if len(s['infoWaiter'][i]) > 0 and s['num_bikes'][i] > 0:
        schedule(Event(now(),i,j, start_riding), fev)

In [ ]:
def arrived_endpoint(s,i,j, fev):
    s['num_bikes'][j] = s['num_bikes'].get(j,0) + 1 
    if len(s['infoWaiter'].get(j,[])) > 0 and s['num_bikes'].get(j,0) > 0:
         schedule(Event(now(),j, s['infoWaiter'][j][0][0], start_riding), fev)

In [ ]:
randomij = randomGenerator()
timeList = [t for t in range(R)] 
import numpy as np

lambda_rate = 2.38
interarrival_times = np.random.exponential(scale=1/lambda_rate, size=R)
arrival_times = np.cumsum(interarrival_times)

In [ ]:
#demo_simulation
state = initial_state()
event_list = FutureEventList()
for k in range(R):
    i, j = randomij[k]
    t = arrival_times[k]
    schedule(Event(t, i, j, arrived_startpoint), event_list)

simulate(state, event_list)

[98.11158798283262, 7.577099162944016]

In [ ]:
# Simulate 30 times
def final_simulator():
  success_rate_list = []
  wait_time_list = []
  MaxWaiter = {} #max wai
  for x in range(30):
    randomij = randomGenerator()
    timeList = [t for t in range(R)] #우선은 그냥 1 분에 1명씩 라이더가 스테이션에 도착하는거로 설정, λ = 2.38 를 이용해서 바뀌줘야함
    import numpy as np

    lambda_rate = 2.38
    interarrival_times = np.random.exponential(scale=1/lambda_rate, size=R)
    arrival_times = np.cumsum(interarrival_times)
    state = initial_state(10)
    event_list = FutureEventList()
    for k in range(R):
      i, j = randomij[k]
      t = arrival_times[k]
      schedule(Event(t, i, j, arrived_startpoint), event_list)
    res = simulate(state, event_list)
    success_rate_list.append(res[0])
    wait_time_list.append(res[1])

  import numpy as np
  from scipy.stats import t
  inputArr = success_rate_list
  sample_mean = np.mean(inputArr)
  sample_std = np.std(inputArr, ddof=1)

  df = len(inputArr) - 1
  alpha = 0.1
  t_crit = t.ppf(1 - alpha / 2, df)

  margin_of_error = t_crit * sample_std / np.sqrt(len(inputArr))

  lower_bound = sample_mean - margin_of_error
  upper_bound = sample_mean + margin_of_error

  print(f"The 90% confidence interval is ({lower_bound:.2f}, {upper_bound:.2f})")
  inputArr = wait_time_list
  sample_mean = np.mean(inputArr)
  sample_std = np.std(inputArr, ddof=1)

  df = len(inputArr) - 1
  alpha = 0.1
  t_crit = t.ppf(1 - alpha / 2, df)

  margin_of_error = t_crit * sample_std / np.sqrt(len(inputArr))

  lower_bound = sample_mean - margin_of_error
  upper_bound = sample_mean + margin_of_error

  print(f"The 90% confidence interval is ({lower_bound:.2f}, {upper_bound:.2f})")

final_simulator()

The 90% confidence interval is (97.56, 97.94)
The 90% confidence interval is (8.28, 10.30)


In [ ]:
def idealized_simulator():
  state = initial_state(1) # set the initial state with one bike per station
  start_bikes = dict(state['num_bikes'])
  c = 0 #looping count variable. Even though 100% success,it checks more unil while loop breaks, 그리고 라이딩을 실패한경우가 있으면 C 다시 리셋.
  while c < 10:
    randomij = randomGenerator()
    timeList = [t for t in range(R)]
    import numpy as np
    lambda_rate = 2.38
    interarrival_times = np.random.exponential(scale=1/lambda_rate, size=R)
    arrival_times = np.cumsum(interarrival_times)
    event_list = FutureEventList()
    for k in range(R):
      i, j = randomij[k]
      t = arrival_times[k]
      schedule(Event(t, i, j, arrived_startpoint), event_list)
    state['num_bikes'] = dict(start_bikes)
    res = simulate(state, event_list)
    for i, waiter in state['infoWaiter'].items():
      if len(waiter) > 0:
        start_bikes[i] += 1
        c=0
    state = initial_state()  # # after simulating, reset 'infoWaiter'and 'num_bikes'
    c+=1
    print(start_bikes)
    print(res)
  print(len(start_bikes))
  return start_bikes

idealized_simulator()
    

{'South Waterfront Walkway - Sinatra Dr & 1 St': 2, 'Grove St PATH': 2, 'Hoboken Terminal - Hudson St & Hudson Pl': 2, 'Hoboken Terminal - River St & Hudson Pl': 2, 'Newport Pkwy': 2, 'City Hall - Washington St & 1 St': 2, 'Newport PATH': 2, '12 St & Sinatra Dr N': 1, 'Hoboken Ave at Monmouth St': 2, 'Marin Light Rail': 2, 'Hamilton Park': 2, '14 St Ferry - 14 St & Shipyard Ln': 2, 'Liberty Light Rail': 1, 'Columbus Dr at Exchange Pl': 2, 'Harborside': 2, '11 St & Washington St': 2, 'Washington St': 2, 'Sip Ave': 2, 'Hudson St & 4 St': 2, '8 St & Washington St': 2, 'Madison St & 1 St': 2, 'City Hall': 2, 'Warren St': 2, 'Newark Ave': 1, 'Columbus Park - Clinton St & 9 St': 2, 'Grand St & 14 St': 2, 'Church Sq Park - 5 St & Park Ave': 2, 'Columbus Drive': 2, 'Van Vorst Park': 2, 'Clinton St & Newark St': 2, 'Grand St': 2, 'Paulus Hook': 2, 'Manila & 1st': 2, '9 St HBLR - Jackson St & 8 St': 2, 'Bloomfield St & 15 St': 2, '4 St & Grand St': 2, '7 St & Monroe St': 2, 'JC Medical Center': 

{'South Waterfront Walkway - Sinatra Dr & 1 St': 39,
 'Grove St PATH': 30,
 'Hoboken Terminal - Hudson St & Hudson Pl': 36,
 'Hoboken Terminal - River St & Hudson Pl': 38,
 'Newport Pkwy': 30,
 'City Hall - Washington St & 1 St': 35,
 'Newport PATH': 33,
 '12 St & Sinatra Dr N': 28,
 'Hoboken Ave at Monmouth St': 28,
 'Marin Light Rail': 28,
 'Hamilton Park': 29,
 '14 St Ferry - 14 St & Shipyard Ln': 24,
 'Liberty Light Rail': 26,
 'Columbus Dr at Exchange Pl': 28,
 'Harborside': 29,
 '11 St & Washington St': 27,
 'Washington St': 25,
 'Sip Ave': 25,
 'Hudson St & 4 St': 26,
 '8 St & Washington St': 28,
 'Madison St & 1 St': 28,
 'City Hall': 25,
 'Warren St': 24,
 'Newark Ave': 22,
 'Columbus Park - Clinton St & 9 St': 24,
 'Grand St & 14 St': 25,
 'Church Sq Park - 5 St & Park Ave': 28,
 'Columbus Drive': 25,
 'Van Vorst Park': 23,
 'Clinton St & Newark St': 24,
 'Grand St': 24,
 'Paulus Hook': 23,
 'Manila & 1st': 23,
 '9 St HBLR - Jackson St & 8 St': 23,
 'Bloomfield St & 15 St': 2